In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import joblib

Aim for this notebook is to extract **'S1501_C01_059E':'Median_earnings_over_25_1_year'** column from Education Census Data column for all the years between 2006 and 2021

***

- [ ] Download US Census Data for all years (2010 to 2021 incl)
- [ ] Compile a dataframe as demonstrated below, median_1_y_earnings
    - Loop through the CVS Files
    - Reformat Census Tract number from Geo_ID
        - Loop through each Census Tract
        - Create a new row input recording the year and the median earnings
- [ ] Export 'median_1_y_earnings' DataFrame



| Census Tract | Median Earnings | Year |
| --- | --- | --- |
| xxxx | xxxx | 2019 |
| --- | --- | --- |
| xxxx | xxxx | 2015 |

***

In [2]:
edu_2021=pd.read_csv('../data/raw/education/ACSST5Y2021.S1501-Data.csv')
edu_2020=pd.read_csv('../data/raw/education/ACSST5Y2020.S1501-Data.csv')
edu_2019=pd.read_csv('../data/raw/education/ACSST5Y2019.S1501-Data.csv')
edu_2018=pd.read_csv('../data/raw/education/ACSST5Y2018.S1501-Data.csv')
edu_2017=pd.read_csv('../data/raw/education/ACSST5Y2017.S1501-Data.csv')
edu_2016=pd.read_csv('../data/raw/education/ACSST5Y2016.S1501-Data.csv')
edu_2015=pd.read_csv('../data/raw/education/ACSST5Y2015.S1501-Data.csv')
edu_2014=pd.read_csv('../data/raw/education/ACSST5Y2014.S1501-Data.csv')
edu_2013=pd.read_csv('../data/raw/education/ACSST5Y2013.S1501-Data.csv')
edu_2012=pd.read_csv('../data/raw/education/ACSST5Y2012.S1501-Data.csv')
edu_2011=pd.read_csv('../data/raw/education/ACSST5Y2011.S1501-Data.csv')
edu_2010=pd.read_csv('../data/raw/education/ACSST5Y2010.S1501-Data.csv')

#Save all dataframes into a list
edu_all=[edu_2021,edu_2020,edu_2019,edu_2018,edu_2017,edu_2016,edu_2015,edu_2014,edu_2013,edu_2012,edu_2011,edu_2010]

edu_backup=edu_all.copy()

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/education/ACSST5Y2021.S1501-Data.csv'

In [8]:
print(f'Shape of one dataframe is {edu_2021.shape}')

Shape of one dataframe is (1333, 1539)


***
* [ ] First Step : Feature Selection

In [19]:
#List of columns that are of interest for this investigation
edu_col_di_extra={'Census_Tract':'Census_Tract','S1501_C01_006E':'Population_over_25','S1501_C01_014E': 'High_school_plus_over_25',
            'S1501_C01_015E':'Bachelors_plus_over_25','S1501_C01_059E':'Median_earnings'}

In [21]:
for edu in edu_all:
    edu['GEO_ID']=edu['GEO_ID'].str[-6:]
    edu.rename(columns={'GEO_ID':'Census_Tract'},inplace=True)

KeyError: 'GEO_ID'

In [22]:
#List of columns that are of interest for this investigation
edu_col_di_extra={'Census_Tract':'Census_Tract','S1501_C01_006E':'Population_over_25','S1501_C01_014E': 'High_school_plus_over_25',
            'S1501_C01_015E':'Bachelors_plus_over_25','S1501_C01_059E':'Median_earnings'}

edu_all[7:]=[edu.rename(columns={'S1501_C01_032E':'S1501_C01_059E'}) for edu in edu_all[7:]]

#edu_col_di={'Census_Tract':'Census_Tract','S1501_C01_059E':'Median_earnings'}

In [24]:
def column_selector(df):

    #remove first row
    df=df.iloc[1:,:]

    df_temp=df[edu_col_di_extra.keys()]
    df_temp=df_temp.rename(columns=edu_col_di_extra)

    for col in list(df_temp.columns):
        df_temp[col]=pd.to_numeric(df_temp[col], errors='coerce')

    #convert to NaN if still string

    return df_temp


In [25]:
edu_all_temp=[column_selector(df) for df in edu_all]

***

**Let's import the dataframes to convert census tract formats (created in PERMITS_CENSUS_TRACTS)**

***Part 1 Conversion DF***

In [396]:
#Import Part 1 Census Tract Dictionary 
census_tract_map=pd.read_csv("../data/interim/di_part1.csv").astype('int32')

census_tract_map.head();

census_tract_di={row['REF']:row['Census_Tract_Match'] for i,row in census_tract_map.iterrows()}

**Further Actions**:
- [ ] Extract all Census Tract Numbers for Education Census Data
- [ ] Compare to the Census Tract Numbers from Permit
- [ ] Check if there is Education Data input for each tract from the Permit list
- [ ] Then filter the Education data census tracts to only include the ones we need

In [397]:
#list of all census tract numbers from Education
edu_tracts_li=[]
for edu in edu_all:
    for item in list(edu['Census_Tract'])[1:]:
        edu_tracts_li.append(int(item))

# selecting the distinct tract numbers
edu_tracts_li=list(set(edu_tracts_li))



#tract numbers from Permit Dataset
permit_tracts=[i for i in pd.read_csv('../data/interim/list.csv',names=['Tracts']).astype(int)['Tracts']]

missing=[i for i in permit_tracts if i not in edu_tracts_li]
print(f'The following tracts are still missing - {missing}')

present=[i for i in edu_tracts_li if i in permit_tracts]

print('\n')
print('mathces between all edu & permits')
print(f"{len(present)}")
print(f"{len(permit_tracts)}")

The following tracts are still missing - []


mathces between all edu & permits
809
809


Check for each year how many census tracts are present
- [ ] Loop through each df
- [ ] Extract a list of census tract - in the format ('edu_tracts=[int(i) for i in list(edu['Census_Tract'])[1:]]')
- [ ] Check for portion of present - in the format (Check for the present=[i for i in edu_tracts if i in permit_tracts])
- [ ] Save as tuples in a new list

In [398]:
num_of_present=[]
year=2021
for edu in edu_all:
    edu_tracts=[int(i) for i in list(edu['Census_Tract'])[1:]]
    present=len([i for i in permit_tracts if i in edu_tracts])
    #checking how many of the education tracts are also in permit_tracrs
    num_of_present.append((year, present))
    year-=1

num_of_present

[(2021, 809),
 (2020, 809),
 (2019, 797),
 (2018, 797),
 (2017, 797),
 (2016, 797),
 (2015, 797),
 (2014, 797),
 (2013, 797),
 (2012, 797),
 (2011, 797),
 (2010, 797)]

Why could the new tracts be created?

In [399]:
edu_tracts_2020=set([int(i) for i in list(edu_2020['Census_Tract'])[1:]])

edu_tracts_2019=set([int(i) for i in list(edu_2019['Census_Tract'])[1:]])

permit_tracts=set(permit_tracts)

In [400]:
#new 2020 tracts (tracts present in permits dataframe that were only added in 2020)
new_tracts_missing=((edu_tracts_2020).symmetric_difference(edu_tracts_2019)).intersection(permit_tracts)
print(new_tracts_missing)

{460800, 320101, 320102, 630600, 612200, 380600, 490200, 330101, 330102, 330103, 844600, 844700}


In [16]:
#CONVERT OLD TO NEW
tract_changes_2020={
            #new_to_old
            '320101':'320100','320102':'320100',
            '330101':'330100','330102':'330100','330103':'330100',
            #old_to_new
            '835900':'380600','380700':'380600',
            '490300':'490200','490400':'490200',
            '460700':'460800','460600':'460800',
            '611000':'612200','611100':'612200',
            '630200':'630600','630100':'630600',
            '380700':'844600','835700':'844600',
            '841400':'844700','841600':'844700'}

In [402]:
#let's also update the permit tracts
permit_tracts_li=[]
for tract in permit_tracts:
    if tract in [int(i) for i in tract_changes_2020.keys()]:
        permit_tracts_li.append(tract_changes_2020[str(tract)])
    else:
        permit_tracts_li.append(tract)

In [17]:
joblib.dump(tract_changes_2020, '../data/interim/tract_changes_2020.pkl')
#di_test=joblib.load('../data/interim/tract_changes_2020.pkl')

['../data/interim/tract_changes_2020.pkl']

In [421]:
edu_all_temp[1]

,Census_Tract,Population_over_25,High_school_plus_over_25,Bachelors_plus_over_25,Median_earnings
1,10100,3443,2948,1725,41424.0
2,10201,4905,4125,1700,35750.0
3,10202,2184,1812,790,31212.0
4,10300,5151,4575,2291,39812.0
5,10400,2742,2649,1643,42526.0
...,...,...,...,...,...
1328,844600,1501,1304,277,26414.0
1329,844700,972,783,143,29896.0
1330,980000,0,0,0,NaN
1331,980100,0,0,0,NaN


In [ ]:
#NEED TO WRITE A FUNCTION THAT WILL UPDATE THE CENSUS TRACTS & AVERAGE FOR THE SAME TRACTS

def census_tract_converted(df):

    df['Census_Tract']=df['Census_Tract'].replace(tract_changes_2020)
    df=df[['Census_Tract','Median_earnings_over_25_1_year']].groupby('Census_Tract').mean().reset_index()

    return df

In [1]:
year=2021
edu_combined=pd.DataFrame()
edu_update=edu_all_temp[0]
for edu in edu_all_temp[1:]:
    print(year)
    edu_combined=pd.merge(edu_update,edu,on='Census_Tract',how='outer',suffixes=('_'+str(year),'_'+str(year-1)))
    edu_update=edu_combined.copy()
    year-=1
    #selecting earnings column

NameError: name 'pd' is not defined

In [424]:
edu_combined['Census_Tract']=edu_combined['Census_Tract'].replace(tract_changes_2020)

In [425]:
edu_combined

,Census_Tract,Population_over_25_2021,High_school_plus_over_25_2021,Bachelors_plus_over_25_2021,Median_earnings_2021,Population_over_25_2020,High_school_plus_over_25_2020,Bachelors_plus_over_25_2020,Median_earnings_2020,Population_over_25_2019,...,Bachelors_plus_over_25_2012,Median_earnings_2012,Population_over_25_2011,High_school_plus_over_25_2011,Bachelors_plus_over_25_2011,Median_earnings_2011,Population_over_25_2010,High_school_plus_over_25_2010,Bachelors_plus_over_25_2010,Median_earnings_2010
0,10100,3285.0,2886.0,1678.0,43829.0,3443.0,2948.0,1725.0,41424.0,3242.0,...,29.0,30455.0,3182.0,85.6,30.0,31860.0,3175.0,82.3,35.0,32631.0
1,10201,4931.0,4230.0,1616.0,36176.0,4905.0,4125.0,1700.0,35750.0,5038.0,...,34.5,29688.0,4348.0,81.1,34.4,29346.0,3669.0,82.4,34.0,30931.0
2,10202,2178.0,1769.0,792.0,30365.0,2184.0,1812.0,790.0,31212.0,2003.0,...,34.3,27299.0,1999.0,78.1,27.1,30408.0,2125.0,76.0,23.2,30136.0
3,10300,5092.0,4435.0,2216.0,46552.0,5151.0,4575.0,2291.0,39812.0,5200.0,...,30.3,36772.0,4725.0,81.5,30.7,40144.0,4838.0,79.2,30.2,40259.0
4,10400,2510.0,2437.0,1480.0,44372.0,2742.0,2649.0,1643.0,42526.0,2649.0,...,59.3,38242.0,2675.0,90.1,52.0,41913.0,2717.0,90.1,50.2,39303.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,829901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6867.0,...,34.0,44018.0,6503.0,94.9,33.4,41989.0,6304.0,94.7,28.9,40291.0
1359,835700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1360,835900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2167.0,...,22.2,32743.0,1997.0,73.1,22.1,33362.0,1848.0,74.7,21.4,30386.0
1361,841400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,619.0,...,5.7,24200.0,784.0,84.6,7.9,23807.0,787.0,87.8,11.3,24777.0


In [426]:
edu_combined=edu_combined.groupby('Census_Tract').mean().reset_index()

In [427]:
edu_combined=edu_combined[edu_combined['Census_Tract'].isin(permit_tracts_li)].reset_index(drop=True)

In [430]:
import re

In [432]:
edu_combined=edu_combined.replace(0,np.NaN)

In [433]:
edu_combined.isna().sum()

Census_Tract                       0
Population_over_25_2021            3
High_school_plus_over_25_2021      3
Bachelors_plus_over_25_2021        3
Median_earnings_2021               8
Population_over_25_2020            3
High_school_plus_over_25_2020      3
Bachelors_plus_over_25_2020        3
Median_earnings_2020               7
Population_over_25_2019           10
High_school_plus_over_25_2019     10
Bachelors_plus_over_25_2019       10
Median_earnings_2019              10
Population_over_25_2018           10
High_school_plus_over_25_2018     10
Bachelors_plus_over_25_2018       10
Median_earnings_2018              10
Population_over_25_2017           10
High_school_plus_over_25_2017    804
Bachelors_plus_over_25_2017      804
Median_earnings_2017              10
Population_over_25_2016           10
High_school_plus_over_25_2016    804
Bachelors_plus_over_25_2016      804
Median_earnings_2016              10
Population_over_25_2015           10
High_school_plus_over_25_2015    804
B

In [463]:

edu_earnings=edu_combined.loc[:,[bool(re.search(r'^Median',col)) for col in edu_combined.columns]]

In [465]:
edu_earnings=pd.concat([edu_combined['Census_Tract'],edu_earnings],axis=1)

In [468]:
edu_earnings.isna().sum()

Census_Tract              0
Median_earnings_2021      8
Median_earnings_2020      7
Median_earnings_2019     10
Median_earnings_2018     10
Median_earnings_2017     10
Median_earnings_2016     10
Median_earnings_2015    653
Median_earnings_2014     10
Median_earnings_2013     10
Median_earnings_2012     10
Median_earnings_2011     10
Median_earnings_2010     11
dtype: int64

In [500]:
edu_earnings['Median_earnings_2015']=edu_earnings['Median_earnings_2015'].apply((lambda x: np.mean(edu_earnings[['Median_earnings_2016','Median_earnings_2014']],axis=1) if x!=np.NaN else x)).iloc[0,:]

In [501]:
edu_earnings.to_csv('../data/interim/edu_earnings.csv')